# Підготовка даних

Google Forms зберіг відповіді користувачів в Excel-файл, але перед тим як аналізувати ці дані, їх слід обробити: об'єднати в один датасет, викинути "сміття", заповнити пропуски і т.д.

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
# прочитати зібраний датасет (Excel файл містить 6 сторінок)
dfs = pd.read_excel('data/Survey Responses.xlsx', sheet_name=None)

# вивести назви сторінок
dfs.keys()

dict_keys(['Контрольна група', 'Менше блоків питань', 'Дружнє опитування', 'Більше картинок', 'Гарний дизайн', 'Сенсація'])

Багато користувачів залишили свою пошту. Згоди на викладання її в публічний доступ вони не давали, тому я приховуватиму електронні адреси.  

In [3]:
# вивести верхні записи в яких не видно пошту
dfs["Контрольна група"][dfs["Контрольна група"].iloc[:, -1].isnull()].head()

,Timestamp,Почати опитування?,До якої вікової групи Ви належите?,Яка Ваша стать?,Ваш поточний рівень доходу?,Чи є у Вас діти?,"В якому регіоні України Ви прожили найбільше часу за все життя, або вважаєте своїм рідним?",Хочете продовжити опитування?,"Яка соціальна мережа/платформа з переліку Вам найбільше подобається, Ви проводите там найбільше часу?",Як Ви оцінюєте свій рівень володіння англійською мовою?,...,Епатажна особистість,Найбільше бажання,Гумористична передача,Фентезі,Квиток на,На вечерю,Улюблений персонаж,Хочете продовжити опитування?.5,Можете залишити свій коментар ✍️,"Залиште свій e-mail, якщо Вам цікаві результати опитування, протягом тижня я зроблю розсилку з аналітикою 📧"
2,2024-03-31 19:47:35.709,так,до 18 років,жінка,зараз не працюю,ні,Центр,так,TikTok,2,...,інше,пристрасне кохання,Дізель Шоу,інше,інше,піцца,інше,так,NaN,NaN
3,2024-04-01 19:09:15.476,так,18-25 років,жінка,до 20 тис.,ні,Захід,так,Instagram,4,...,інше,пристрасне кохання,95 Квартал,Гаррі Поттер,концерт Ріанни,суші,Патрік,так,NaN,NaN
4,2024-04-01 19:15:56.985,так,18-25 років,жінка,більше 40 тис.,ні,Центр,так,Telegram,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2024-04-01 19:18:25.333,так,18-25 років,жінка,до 20 тис.,ні,Центр,так,Telegram,2,...,Михайло Поплавський,чемодан грошей,Дізель Шоу,Гаррі Поттер,концерт Ріанни,суші,Губка Боб,так,NaN,NaN
7,2024-04-01 20:00:27.354,так,до 18 років,чоловік,зараз не працюю,ні,Центр,так,Telegram,2,...,Каньє Вест,пристрасне кохання,95 Квартал,Гаррі Поттер,інше,піцца,інше,так,NaN,NaN


Таблиця **"Менше блоків питань"** має менше колонок, ніж решта таблиць, бо в цьому опитуванні я "злив" по декілька блоків питань в один, відповідно менше раз запитував у користувача, чи хоче він продовжити опитування. Для того, щоб можна було об'єднати таблиці, вони мають містити однакову кількість колонок, які мають однакові назви.  
Додамо у відповідні місця колонки, яких не вистачає, продублювавши значення з тих колонок, які є.    
В Excel якщо декілька колонок мають однакові назви, то перша така назва залишається як є, а до решти дописуються закінчення ".1", ".2" і т.д. щоб не було повторів і до колонок можна було звертатись по їх назві.

In [4]:
# додати колонки
dfs['Менше блоків питань'].insert(19, 'Хочете продовжити опитування?.3', dfs['Менше блоків питань'].iloc[:, 7])
dfs['Менше блоків питань'].insert(40, 'Хочете продовжити опитування?.4', dfs['Менше блоків питань'].iloc[:, 32])
dfs['Менше блоків питань'].insert(50, 'Хочете продовжити опитування?.5', dfs['Менше блоків питань'].iloc[:, 32])

Таблиця **"Дружнє опитування"** має трохи інші назви колонок (там я звертаюсь до користувача по-дружньому на "ти").   
В таблиці **"Менше блоків питань"** колонки, де запитується, чи хоче користувач продовжити, мають непрямий порядок (тобто закінчення ".1", ".2" і т.д. йдуть не підряд).  
В таблиці **"Сенсація"** відрізняється назва колонки, де я пропоную користувачу залишити пошту.  
Зроблю так, щоб колонки в цих таблицях називались так само, як в **"Контрольній групі"**.

In [5]:
# зробити однакові назви колонок в таблицях
dfs['Менше блоків питань'] = dfs['Менше блоків питань'].rename(columns=dict(zip(dfs['Менше блоків питань'].columns, dfs["Контрольна група"].columns)))
dfs['Дружнє опитування'] = dfs['Дружнє опитування'].rename(columns=dict(zip(dfs['Дружнє опитування'].columns, dfs["Контрольна група"].columns)))
dfs['Сенсація'] = dfs['Сенсація'].rename(columns=dict(zip(dfs['Сенсація'].columns, dfs["Контрольна група"].columns)))

Об'єдную таблиці в одну.

In [6]:
# додати колонку, яка позначатиме до якого типу опитування відноситься запис
for title, df in dfs.items():
    df['Опитування'] = title

# об'єднати датафрейми
merged_df = pd.concat(list(dfs.values()), ignore_index=True).reindex(columns=dfs['Контрольна група'].columns)

# відсортувати записи в хронологічному порядку
merged_df = merged_df.sort_values(by='Timestamp').reset_index(drop=True)

# вивести верхні записи у яких не видно пошту
merged_df[merged_df.iloc[:, -2].isnull()].head()

,Timestamp,Почати опитування?,До якої вікової групи Ви належите?,Яка Ваша стать?,Ваш поточний рівень доходу?,Чи є у Вас діти?,"В якому регіоні України Ви прожили найбільше часу за все життя, або вважаєте своїм рідним?",Хочете продовжити опитування?,"Яка соціальна мережа/платформа з переліку Вам найбільше подобається, Ви проводите там найбільше часу?",Як Ви оцінюєте свій рівень володіння англійською мовою?,...,Найбільше бажання,Гумористична передача,Фентезі,Квиток на,На вечерю,Улюблений персонаж,Хочете продовжити опитування?.5,Можете залишити свій коментар ✍️,"Залиште свій e-mail, якщо Вам цікаві результати опитування, протягом тижня я зроблю розсилку з аналітикою 📧",Опитування
0,2024-03-29 20:07:30.607,так,18-25 років,чоловік,зараз не працюю,ні,Південь,ні,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Сенсація
2,2024-03-30 20:28:35.284,так,18-25 років,чоловік,зараз не працюю,ні,Захід,так,YouTube,3.0,...,інше,інше,інше,інше,піцца,Губка Боб,так,На всі питання старався відповідати чесно (як ...,NaN,Сенсація
5,2024-03-30 22:04:38.078,так,36-60 років,жінка,більше 40 тис.,так,Захід,так,Instagram,3.0,...,пристрасне кохання,інше,Володар перснів,інше,суші,інше,так,"Є питання, де хотілося б обрати кілька варіантів",NaN,Дружнє опитування
6,2024-03-31 10:27:42.537,так,26-35 років,чоловік,30-40 тис.,ні,Схід,так,Telegram,3.0,...,чемодан грошей,інше,Володар перснів,інше,суші,інше,так,NaN,NaN,Сенсація
7,2024-03-31 19:26:45.802,так,18-25 років,чоловік,більше 40 тис.,ні,ніколи не жив в Україні 🥲,так,Telegram,4.0,...,чемодан грошей,Дізель Шоу,Гаррі Поттер,фінал Ліги Чемпіонів,суші,Губка Боб,так,NaN,NaN,Більше картинок


Далі я буду змінювати датафрейм (видаляти/додавати певні колонки та рядки).  
Перш ніж змінювати оригінальні дані, слід розглянути конверсії з одного етапу опитування в інший.

In [7]:
# дані про переходи користувачів по типам опитування з сайту LINKLY
survey_clicks = {
    "Контрольна група": 79,
    "Дружнє опитування": 80,
    "Менше блоків питань": 70,
    "Сенсація": 60,
    "Більше картинок": 69,
    "Гарний дизайн": 70
}

# назви типів опитувань
survey_names = list(dfs.keys())

# кількість "кліків" на кожен тип опитування
clicks = [survey_clicks[survey_name] for survey_name in survey_names]

# кількість користувачів, які почали опитування для кожного типу
start_survey = [merged_df[merged_df['Опитування'] == survey_name]['Почати опитування?'].value_counts().get('так', 0) for survey_name in survey_names]

# кількість користувачів, які перейшли на наступний етап для кожного етапу воронки користувача
funnel_stage1 = [merged_df[merged_df['Опитування'] == survey_name]['Хочете продовжити опитування?'].value_counts().get('так', 0) for survey_name in survey_names]
funnel_stage2 = [merged_df[merged_df['Опитування'] == survey_name]['Хочете продовжити опитування?.1'].value_counts().get('так', 0) for survey_name in survey_names]
funnel_stage3 = [merged_df[merged_df['Опитування'] == survey_name]['Хочете продовжити опитування?.2'].value_counts().get('так', 0) for survey_name in survey_names]
funnel_stage4 = [merged_df[merged_df['Опитування'] == survey_name]['Хочете продовжити опитування?.3'].value_counts().get('так', 0) for survey_name in survey_names]
funnel_stage5 = [merged_df[merged_df['Опитування'] == survey_name]['Хочете продовжити опитування?.4'].value_counts().get('так', 0) for survey_name in survey_names]
funnel_stage6 = [merged_df[merged_df['Опитування'] == survey_name]['Хочете продовжити опитування?.5'].value_counts().get('так', 0) for survey_name in survey_names]
emails = [merged_df[merged_df['Опитування'] == survey_name]['Залиште свій e-mail, якщо Вам цікаві результати опитування, протягом тижня я зроблю розсилку з аналітикою 📧'].count() for survey_name in survey_names]

# створити датафрейм, який містить дані про конверсії
conversions_df = pd.DataFrame(
    np.array([
        clicks,
        start_survey,
        funnel_stage1,
        funnel_stage2,
        funnel_stage3,
        funnel_stage4,
        funnel_stage5,
        funnel_stage6,
        emails
    ]),
    columns=survey_names,
    index=["Кліки", "Кліки => Знайомство", "Знайомство => Асортимент", "Асортимент => Дихотомія", "Дихотомія => Аукціон", "Аукціон => Мемологія", "Мемологія => Бліц", "Бліц => Фінал", "Фінал => Пошта"]
)

# подивитись як виглядає датафрейм
conversions_df

,Контрольна група,Менше блоків питань,Дружнє опитування,Більше картинок,Гарний дизайн,Сенсація
Кліки,79,70,80,69,70,60
Кліки => Знайомство,32,40,48,37,33,47
Знайомство => Асортимент,32,40,48,36,33,46
Асортимент => Дихотомія,32,40,44,36,32,46
Дихотомія => Аукціон,30,38,42,33,30,42
Аукціон => Мемологія,29,38,41,33,30,42
Мемологія => Бліц,28,38,41,31,30,41
Бліц => Фінал,28,37,41,31,30,40
Фінал => Пошта,15,22,24,20,21,20


In [8]:
# зберегти датасет з конверсіями
conversions_df.to_excel("data/Conversions.xlsx")

"Закодую" електронні пошти респондентів, щоб викласти в публічний доступ датасет з захищеними персональними даними.

In [9]:
def is_valid_email(email):
    """ Функція перевіряє, чи коректно вказана електронна адреса. """
    pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    return re.match(pattern, email) is not None

def encrypt_email(email):
    """ Функція "шифрує" електронну адресу. """
    if pd.notna(email) and is_valid_email(email):
        return "".join(['x' if char not in '@.' else char for char in email])


# "зашифрувати" електронні адреси користувачів
merged_df[dfs['Дружнє опитування'].columns[-2]] = merged_df[dfs['Дружнє опитування'].columns[-2]].apply(encrypt_email)

# подивитись як виглядає датафрейм
merged_df.head()

,Timestamp,Почати опитування?,До якої вікової групи Ви належите?,Яка Ваша стать?,Ваш поточний рівень доходу?,Чи є у Вас діти?,"В якому регіоні України Ви прожили найбільше часу за все життя, або вважаєте своїм рідним?",Хочете продовжити опитування?,"Яка соціальна мережа/платформа з переліку Вам найбільше подобається, Ви проводите там найбільше часу?",Як Ви оцінюєте свій рівень володіння англійською мовою?,...,Найбільше бажання,Гумористична передача,Фентезі,Квиток на,На вечерю,Улюблений персонаж,Хочете продовжити опитування?.5,Можете залишити свій коментар ✍️,"Залиште свій e-mail, якщо Вам цікаві результати опитування, протягом тижня я зроблю розсилку з аналітикою 📧",Опитування
0,2024-03-29 20:07:30.607,так,18-25 років,чоловік,зараз не працюю,ні,Південь,ні,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,Сенсація
1,2024-03-29 21:24:34.496,так,18-25 років,жінка,20-30 тис.,ні,Схід,так,Telegram,2.0,...,чемодан грошей,95 Квартал,Гаррі Поттер,концерт Ріанни,суші,Патрік,так,"Шо це за хуйня? Ну добре, ебала в рот я такие ...",xxxxxxxxxxxx@xxxxx.xxx,Контрольна група
2,2024-03-30 20:28:35.284,так,18-25 років,чоловік,зараз не працюю,ні,Захід,так,YouTube,3.0,...,інше,інше,інше,інше,піцца,Губка Боб,так,На всі питання старався відповідати чесно (як ...,None,Сенсація
3,2024-03-30 20:42:28.672,так,26-35 років,чоловік,більше 40 тис.,ні,Північ,так,Telegram,5.0,...,інше,інше,Володар перснів,інше,суші,інше,так,Не так вже й довго :) Деяким питанням не виста...,xxxxxxxxx@xxxxx.xxx,Контрольна група
4,2024-03-30 21:26:40.839,так,18-25 років,чоловік,зараз не працюю,ні,Центр,так,Telegram,3.0,...,інше,інше,Гаррі Поттер,фінал Ліги Чемпіонів,піцца,Губка Боб,так,NaN,xxxxxxxxxxx@xxxxx.xxx,Сенсація


In [10]:
# зберегти датасет в якому приховані пошти
merged_df.to_excel('data/Public Survey Responses.xlsx', index=False)

Насправді вміст коментаря та електронна адреса мене цікавить не так, як сам факт їхньої наявності. Тому замість цих колонок додам булеві колонки **"є пошта"** та **"є коментар"**.

In [11]:
# створити булеву колонку
merged_df['є коментар'] = merged_df['Можете залишити свій коментар ✍️'].notna()

# видалити колонку з коментарями користувачів
merged_df.drop(columns=['Можете залишити свій коментар ✍️'], inplace=True)

# створити булеву колонку
merged_df['є пошта'] = merged_df['Залиште свій e-mail, якщо Вам цікаві результати опитування, протягом тижня я зроблю розсилку з аналітикою 📧'].notna()

# видалити колонку з електронними адресами користувачів
merged_df.drop(columns=['Залиште свій e-mail, якщо Вам цікаві результати опитування, протягом тижня я зроблю розсилку з аналітикою 📧'], inplace=True)

# подивитись як виглядає датафрейм
merged_df.head()

,Timestamp,Почати опитування?,До якої вікової групи Ви належите?,Яка Ваша стать?,Ваш поточний рівень доходу?,Чи є у Вас діти?,"В якому регіоні України Ви прожили найбільше часу за все життя, або вважаєте своїм рідним?",Хочете продовжити опитування?,"Яка соціальна мережа/платформа з переліку Вам найбільше подобається, Ви проводите там найбільше часу?",Як Ви оцінюєте свій рівень володіння англійською мовою?,...,Найбільше бажання,Гумористична передача,Фентезі,Квиток на,На вечерю,Улюблений персонаж,Хочете продовжити опитування?.5,Опитування,є коментар,є пошта
0,2024-03-29 20:07:30.607,так,18-25 років,чоловік,зараз не працюю,ні,Південь,ні,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Сенсація,False,False
1,2024-03-29 21:24:34.496,так,18-25 років,жінка,20-30 тис.,ні,Схід,так,Telegram,2.0,...,чемодан грошей,95 Квартал,Гаррі Поттер,концерт Ріанни,суші,Патрік,так,Контрольна група,True,True
2,2024-03-30 20:28:35.284,так,18-25 років,чоловік,зараз не працюю,ні,Захід,так,YouTube,3.0,...,інше,інше,інше,інше,піцца,Губка Боб,так,Сенсація,True,False
3,2024-03-30 20:42:28.672,так,26-35 років,чоловік,більше 40 тис.,ні,Північ,так,Telegram,5.0,...,інше,інше,Володар перснів,інше,суші,інше,так,Контрольна група,True,True
4,2024-03-30 21:26:40.839,так,18-25 років,чоловік,зараз не працюю,ні,Центр,так,Telegram,3.0,...,інше,інше,Гаррі Поттер,фінал Ліги Чемпіонів,піцца,Губка Боб,так,Сенсація,False,True


Для аналізу результатів опитування колонки, де я питаю, чи хоче користувач продовжити, не є цінними. Вони знадобляться тільки для підрахунку конверсій.  
Колонка з точним часом, коли була дана відповідь також не є інформативною, бо в різні чати я закидував повідомлення в різні дні, і не знаю точно з якого чату прийшов користувач.  
Також я відкидаю ті записи, де користувач навіть не починав опитування, або зупинився вже після першого блоку.

In [12]:
# відкинути респондентів, які сказали "ні" на перших двох етапах воронки користувача
merged_df = merged_df[merged_df['Почати опитування?'] != 'ні']
merged_df = merged_df[merged_df['Хочете продовжити опитування?'] != 'ні']

# видалити стовпчики, які не потрібні для аналізу
merged_df = merged_df.drop(columns=[
    'Timestamp',
    'Почати опитування?',
    'Хочете продовжити опитування?',
    'Хочете продовжити опитування?.1',
    'Хочете продовжити опитування?.2',
    'Хочете продовжити опитування?.3',
    'Хочете продовжити опитування?.4',
    'Хочете продовжити опитування?.5'
])

До останнього етапу опитування не дійшло трохи більше 10% респондентів. Це не критично багато, але вибірка загалом відносно мала, тому втрачати навіть ці дані не хотілося б.  
Замість цього я заповнюю пропуски "типовими" значеннями: для категоріальних даних - мода по даному типу опитування, для числових - середнє усічене по даному типу опитування (відкидаю по 10% найменших та найбільших значень, бо при огляді даних помітив викиди).

In [13]:
# порахувати кількість пропусків в останньому питанні
count_nan = merged_df['Улюблений персонаж'].isna().sum()
print(f"Всього {count_nan} пропусків ({round(count_nan/len(merged_df) * 100, 2)}% від датафрейму).")

Всього 26 пропусків (11.06% від датафрейму).


Відповідями на деякі питання є число "1" або "2", які насправді позначають категорію (наприклад "Оберіть, яке зображення вам більше подобається: 1 чи 2?"). Але для них автоматично встановлюється тип 'float64'. Переводжу їх явно в тип 'object'.

In [14]:
# перевести колонки в тип 'object' 
for column in merged_df.columns:
    # тільки перераховані колонки мають числовий тип
    if column not in [
        'Футболка в якій Ліонель Мессі грав у переможному фіналі ЛЧ 2014/15',
       'Нова книга Стівена Кінга з автографом автора',
       'Обід з Михайлом Лебігою',
       'Розбита гітара з першого концерту Курта Кобейна',
       'Спаринг з Дмитром Гордоном',
       'Панамка в якій фронтмен гурту Калуш виграв Євробачення',
       'Сережки Емми Уотсон'
    ] and merged_df[column].dtype == 'float64':
        merged_df[column] = merged_df[column].astype('object')

# ця колонка навпаки має тип 'object', переводжу її в числовий тип
merged_df["Спаринг з Дмитром Гордоном"] = merged_df["Спаринг з Дмитром Гордоном"].astype('float64')

In [15]:
# заповнити пропуски типовим значенням для даного типу опитування
df = merged_df.groupby('Опитування').transform(lambda x: x.fillna(x.mode()[0]) if x.dtype != 'float64' else x.fillna(round(x[(x >= x.quantile(0.10)) & (x <= x.quantile(0.90))].mean(), 2)))

# після заповення пропусків зникла колонка "Опитування", але порядок записів лишився той самий
# додаю цю колонку знову
df["Опитування"] = merged_df["Опитування"]

# подивитись як виглядає датафрейм
df.head()

,До якої вікової групи Ви належите?,Яка Ваша стать?,Ваш поточний рівень доходу?,Чи є у Вас діти?,"В якому регіоні України Ви прожили найбільше часу за все життя, або вважаєте своїм рідним?","Яка соціальна мережа/платформа з переліку Вам найбільше подобається, Ви проводите там найбільше часу?",Як Ви оцінюєте свій рівень володіння англійською мовою?,Яке зараз ваше ставлення до пса Патрона?,Як Ви ставитесь до реклами букмекерів/казино?,Як Ви оцінюєте свій стан здоров'я?,...,Епатажна особистість,Найбільше бажання,Гумористична передача,Фентезі,Квиток на,На вечерю,Улюблений персонаж,є коментар,є пошта,Опитування
1,18-25 років,жінка,20-30 тис.,ні,Схід,Telegram,2.0,не впевнений,дуже негативно,5.0,...,Каньє Вест,чемодан грошей,95 Квартал,Гаррі Поттер,концерт Ріанни,суші,Патрік,True,True,Контрольна група
2,18-25 років,чоловік,зараз не працюю,ні,Захід,YouTube,3.0,не впевнений,дуже негативно,4.0,...,інше,інше,інше,інше,інше,піцца,Губка Боб,True,False,Сенсація
3,26-35 років,чоловік,більше 40 тис.,ні,Північ,Telegram,5.0,скоріше позитивне,дуже негативно,4.0,...,Михайло Поплавський,інше,інше,Володар перснів,інше,суші,інше,True,True,Контрольна група
4,18-25 років,чоловік,зараз не працюю,ні,Центр,Telegram,3.0,не впевнений,все рівно,3.0,...,інше,інше,інше,Гаррі Поттер,фінал Ліги Чемпіонів,піцца,Губка Боб,False,True,Сенсація
5,36-60 років,жінка,більше 40 тис.,так,Захід,Instagram,3.0,скоріше позитивне,скоріше негативно,3.0,...,інше,пристрасне кохання,інше,Володар перснів,інше,суші,інше,True,False,Дружнє опитування


В блоці питань про аукціон деякі фантазери вказували суми типу 1 млн. грн, хоча я просив робити "ставку" спираючись на їхні реальні фінансові можливості. Життєвий досвід підказує мені, що серед жителів студмістечок рідко зустрічаються мільйонери, тому краще "обрубати" ці казкові числа, наприклад, на сумі 50000 грн.

In [16]:
# встановити максимальне числове значення 50000
for col in df.columns:
    if df[col].dtype == "float64":
        df[col] = df[col].clip(lower=0, upper=50000)

# подивитись як виглядає датафрейм 
df.head()

,До якої вікової групи Ви належите?,Яка Ваша стать?,Ваш поточний рівень доходу?,Чи є у Вас діти?,"В якому регіоні України Ви прожили найбільше часу за все життя, або вважаєте своїм рідним?","Яка соціальна мережа/платформа з переліку Вам найбільше подобається, Ви проводите там найбільше часу?",Як Ви оцінюєте свій рівень володіння англійською мовою?,Яке зараз ваше ставлення до пса Патрона?,Як Ви ставитесь до реклами букмекерів/казино?,Як Ви оцінюєте свій стан здоров'я?,...,Епатажна особистість,Найбільше бажання,Гумористична передача,Фентезі,Квиток на,На вечерю,Улюблений персонаж,є коментар,є пошта,Опитування
1,18-25 років,жінка,20-30 тис.,ні,Схід,Telegram,2.0,не впевнений,дуже негативно,5.0,...,Каньє Вест,чемодан грошей,95 Квартал,Гаррі Поттер,концерт Ріанни,суші,Патрік,True,True,Контрольна група
2,18-25 років,чоловік,зараз не працюю,ні,Захід,YouTube,3.0,не впевнений,дуже негативно,4.0,...,інше,інше,інше,інше,інше,піцца,Губка Боб,True,False,Сенсація
3,26-35 років,чоловік,більше 40 тис.,ні,Північ,Telegram,5.0,скоріше позитивне,дуже негативно,4.0,...,Михайло Поплавський,інше,інше,Володар перснів,інше,суші,інше,True,True,Контрольна група
4,18-25 років,чоловік,зараз не працюю,ні,Центр,Telegram,3.0,не впевнений,все рівно,3.0,...,інше,інше,інше,Гаррі Поттер,фінал Ліги Чемпіонів,піцца,Губка Боб,False,True,Сенсація
5,36-60 років,жінка,більше 40 тис.,так,Захід,Instagram,3.0,скоріше позитивне,скоріше негативно,3.0,...,інше,пристрасне кохання,інше,Володар перснів,інше,суші,інше,True,False,Дружнє опитування


Останнє, що ще слід виправити: помітив, що в одному з типів опитування написав "Мікеланджело", а в решті "Мікіланджело".  
Уніфікую згадки імені цього персонажа в датасеті.

In [17]:
df["З ким з черепашок-ніндзя Ви себе скоріше асоціюєте?"] = df["З ким з черепашок-ніндзя Ви себе скоріше асоціюєте?"].replace("Мікеланджело", "Мікіланджело")

In [18]:
# зберегти очищений датасет
df.to_excel('data/Cleaned Data.xlsx', index=False)